In [ ]:
# ======================================================================================================
# Experimento 03: Medición de la proyección de los ángulos de los caninos sobre la línea media central
# Se utilizan dos modelos
# 1) Un modelo U-Net para segmentar la forma del canino (máscara)
# 2) Un modelo YoloV5 para detectar la posición de los caninos e incisivos centrales

import cv2
import numpy as np
import torch
import os
import math
import sys
sys.path.append('../src')
from tooth_shape_model_unet import UNet
from torchvision import transforms
from PIL import Image 


# ======================================================================
# Modelo U-Net para segmentación de caninos (obtener la forma del canino, para trazar la línea sobre él)

IMG_HEIGHT = 256
IMG_WIDTH = 256
NUM_CLASSES = 3  # fondo, canino izquierdo, canino derecho

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
shape_unet = UNet(num_classes=NUM_CLASSES).to(device)
shape_unet.load_state_dict(torch.load('models/tooth_shape_unet.pth', map_location=device))
shape_unet.eval()

infer_transform = transforms.Compose([
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
    transforms.ToTensor(),
])

# ======================================================================
# Modelo pre-entrenado (YoloV5) para la detección posicional de caninos e incisivos centrales
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torch.hub.load(
    'ultralytics/yolov5', 
    'custom',
    path='models/tooth_detection.pt',
    force_reload=True
).to(device)

# Umbral de confianza de detección posicional de caninos
CONF_THRESHOLD = 0.85
model.conf = CONF_THRESHOLD
print(f"Umbral de confianza ajustado a: {CONF_THRESHOLD}")

def calculate_line_intersection(line1_p1, line1_p2, line2_p1, line2_p2):
    """
    Calcula el punto de intersección entre dos líneas definidas por dos puntos cada una.
    Si las líneas son paralelas o no se intersectan dentro de los límites de la imagen, devuelve None.
    
    Args:
        line1_p1, line1_p2: Puntos que definen la primera línea
        line2_p1, line2_p2: Puntos que definen la segunda línea
        
    Returns:
        tuple: Coordenadas (x, y) del punto de intersección o None si no hay intersección
    """
    # Línea 1: (x1, y1) a (x2, y2)
    x1, y1 = line1_p1
    x2, y2 = line1_p2
    
    # Línea 2: (x3, y3) a (x4, y4)
    x3, y3 = line2_p1
    x4, y4 = line2_p2
    
    # Calcular denominador para verificar si las líneas son paralelas
    denom = (y4 - y3) * (x2 - x1) - (x4 - x3) * (y2 - y1)
    
    if denom == 0:  # Líneas paralelas
        return None
    
    # Calcular el punto de intersección
    ua = ((x4 - x3) * (y1 - y3) - (y4 - y3) * (x1 - x3)) / denom
    
    # Punto de intersección
    x = x1 + ua * (x2 - x1)
    y = y1 + ua * (y2 - y1)
    
    return (int(x), int(y))

def extend_line_to_boundaries(point1, point2, img_width, img_height, midline_x=None):
    """
    Extiende una línea definida por dos puntos hasta los límites de la imagen o hasta intersectar con la línea media.
    
    Args:
        point1 (tuple): Coordenadas (x, y) del primer punto.
        point2 (tuple): Coordenadas (x, y) del segundo punto.
        img_width (int): Ancho de la imagen.
        img_height (int): Alto de la imagen.
        midline_x (int, opcional): Coordenada x de la línea media. Si se proporciona, la línea se extenderá hasta esta línea.
        
    Returns:
        tuple: Un par de tuplas con las coordenadas de los puntos extendidos (p1_extended, p2_extended).
    """
    x1, y1 = point1
    x2, y2 = point2
    
    # Si los puntos son iguales, no se puede definir una dirección
    if x1 == x2 and y1 == y2:
        return point1, point2
    
    # Calcular la dirección de la línea
    dx = x2 - x1
    dy = y2 - y1
    
    # Si la línea es vertical
    if dx == 0:
        # Extender hasta los bordes superior e inferior
        return (x1, 0), (x1, img_height)
    
    # Calcular la pendiente y el intercepto
    m = dy / dx
    b = y1 - m * x1
    
    # Puntos extendidos
    extended_points = []
    
    # Si hay una línea media definida, calcular la intersección con ella
    if midline_x is not None:
        # Calcular el punto de intersección con la línea media
        y_intersect = m * midline_x + b
        
        # Verificar si la intersección está dentro de los límites de la imagen
        if 0 <= y_intersect <= img_height:
            # Determinar en qué lado de la línea media está el punto original
            if (x1 < midline_x and x2 < midline_x) or (x1 > midline_x and x2 > midline_x):
                # Ambos puntos están en el mismo lado de la línea media
                # Extender hasta la línea media en una dirección
                if x1 < midline_x:
                    extended_points.append((midline_x, int(y_intersect)))
                else:
                    extended_points.append((midline_x, int(y_intersect)))
            elif (x1 < midline_x and x2 > midline_x) or (x1 > midline_x and x2 < midline_x):
                # No es necesario extender hasta la línea media
                pass
    
    # Intersecciones con los bordes de la imagen
    
    # Intersección con y=0 (borde superior)
    if abs(m) > 0.0001:  # No es una línea horizontal
        x_top = (0 - b) / m
        if 0 <= x_top <= img_width:
            extended_points.append((int(x_top), 0))
    
    # Intersección con y=img_height (borde inferior)
    if abs(m) > 0.0001:  # No es una línea horizontal
        x_bottom = (img_height - b) / m
        if 0 <= x_bottom <= img_width:
            extended_points.append((int(x_bottom), img_height))
    
    # Intersección con x=0 (borde izquierdo)
    y_left = b
    if 0 <= y_left <= img_height:
        extended_points.append((0, int(y_left)))
    
    # Intersección con x=img_width (borde derecho)
    y_right = m * img_width + b
    if 0 <= y_right <= img_height:
        extended_points.append((img_width, int(y_right)))
    
    # Si no hay suficientes puntos de intersección, usar los puntos originales
    if len(extended_points) < 2:
        return point1, point2
    
    # Ordenar los puntos extendidos según su distancia desde el punto medio entre p1 y p2
    midpoint = ((x1 + x2) / 2, (y1 + y2) / 2)
    
    # Si estamos del lado izquierdo de la línea media y queremos extender hacia la línea media
    if midline_x is not None and (x1 < midline_x and x2 < midline_x):
        # Encontrar el punto más cercano al borde y el punto más cercano a la línea media
        points_sorted = sorted(extended_points, key=lambda p: p[0])  # Ordenar por coordenada x
        return points_sorted[0], points_sorted[-1]  # El primero es el más a la izquierda, el último es el más a la derecha
    
    # Si estamos del lado derecho de la línea media y queremos extender hacia la línea media
    elif midline_x is not None and (x1 > midline_x and x2 > midline_x):
        # Encontrar el punto más cercano al borde y el punto más cercano a la línea media
        points_sorted = sorted(extended_points, key=lambda p: p[0], reverse=True)  # Ordenar por coordenada x (reverso)
        return points_sorted[0], points_sorted[-1]  # El primero es el más a la derecha, el último es el más a la izquierda
    
    # En otros casos, simplemente usar las dos intersecciones más alejadas entre sí
    else:
        # Calcular todas las combinaciones de distancias entre puntos
        max_dist = 0
        p1_ext, p2_ext = extended_points[0], extended_points[1]
        
        for i in range(len(extended_points)):
            for j in range(i + 1, len(extended_points)):
                dist = math.sqrt((extended_points[i][0] - extended_points[j][0])**2 + 
                                (extended_points[i][1] - extended_points[j][1])**2)
                if dist > max_dist:
                    max_dist = dist
                    p1_ext, p2_ext = extended_points[i], extended_points[j]
        
        return p1_ext, p2_ext


def get_corners(detection, side):
    if side == 'izq':
        return (int(detection['xmax']), int(detection['ymax'])), (int(detection['xmin']), int(detection['ymin']))
    elif side == 'der':
        return (int(detection['xmin']), int(detection['ymax'])), (int(detection['xmax']), int(detection['ymin']))
    else:   
        raise ValueError("Lado no válido. Debe ser 'izq' o 'der'.")


def segment_full_and_crop(orig_img, roi_coords):
    # Segmenta toda la imagen y recorta ROI
    img_rgb = cv2.cvtColor(orig_img, cv2.COLOR_BGR2RGB)
    pil = Image.fromarray(img_rgb)
    x = infer_transform(pil).unsqueeze(0).to(device)
    with torch.no_grad():
        logits = shape_unet(x)
        probs  = torch.softmax(logits, dim=1)[0]  # [3,H,W]

    # si side=='left'
    ch = probs[1] if side=='left' else probs[2]
    heatmap = (ch.cpu().numpy() * 255).astype(np.uint8)
    h, w = orig_img.shape[:2]
    heatmap_full = cv2.resize(heatmap, (w, h), interpolation=cv2.INTER_LINEAR)
    cv2.imwrite(f"debug/heat_full_{side}.png", heatmap_full)
    chan = probs[1] if side=='left' else probs[2]
    mask = (chan.cpu().numpy() > 0.03).astype(np.uint8) * 255
    # redimensionar a full-res
    h,w = orig_img.shape[:2]
    mask_full = cv2.resize(mask,(w,h),interpolation=cv2.INTER_NEAREST)
    # recortar
    # x1,y1,x2,y2 = roi_coords
    return mask_full


def calculate_angle(line_p1, line_p2, vertical_line_x):
    """
    Calcula el ángulo entre una línea definida por dos puntos y una línea vertical.
    
    Args:
        line_p1 (tuple): Primer punto de la línea.
        line_p2 (tuple): Segundo punto de la línea.
        vertical_line_x (int): Coordenada x de la línea vertical.
        
    Returns:
        float: Ángulo en grados entre las líneas.
    """
    # Verificar que los puntos no sean iguales
    if line_p1[0] == line_p2[0] and line_p1[1] == line_p2[1]:
        return 0  # No se puede calcular el ángulo si los puntos son iguales
    
    # Vector de la línea
    vector_line = (line_p2[0] - line_p1[0], line_p2[1] - line_p1[1])
    
    # Vector de la línea vertical (0, 1) normalizado
    vector_vertical = (0, 1)
    
    # Calcular el ángulo entre los vectores usando el producto punto
    # Normalizar los vectores
    magnitude_line = math.sqrt(vector_line[0]**2 + vector_line[1]**2)
    
    if magnitude_line == 0:
        return 0
    
    unit_vector_line = (vector_line[0] / magnitude_line, vector_line[1] / magnitude_line)
    
    # Producto punto de los vectores unitarios
    dot_product = unit_vector_line[0] * vector_vertical[0] + unit_vector_line[1] * vector_vertical[1]
    
    # Asegurarse de que el producto punto esté en el rango [-1, 1]
    dot_product = max(-1.0, min(1.0, dot_product))
    
    # Calcular el ángulo en radianes y convertirlo a grados
    angle_rad = math.acos(dot_product)
    angle_deg = math.degrees(angle_rad)
    
    # Determinar la dirección del ángulo (positivo o negativo)
    # Si el punto p2 está a la derecha de la línea vertical, el ángulo es positivo
    # Si está a la izquierda, el ángulo es negativo
    direction = 1 if (line_p1[0] < vertical_line_x and line_p2[0] > vertical_line_x) or \
                    (line_p1[0] > vertical_line_x and line_p2[0] < vertical_line_x and line_p1[1] > line_p2[1]) else -1
    
    # Ajustar el ángulo según el cuadrante
    if unit_vector_line[0] < 0:
        angle_deg = 180 - angle_deg
    
    # Asegurarse de que el ángulo esté entre 0 y 180 grados
    if angle_deg > 90:
        angle_deg = 180 - angle_deg
        
    return angle_deg * direction

def process_and_draw_canine(det, orig_img, side, inc_center_x=None):
    coords = (int(det['xmin']), int(det['ymin']), int(det['xmax']), int(det['ymax']))
    # Extraemos Cordenadas
    x1, y1, x2, y2 = coords
    roi = orig_img[coords[1]:coords[3], coords[0]:coords[2]]
    # 1) Segmentar
    mask_roi = segment_full_and_crop(roi, coords)
    cv2.imwrite(f"debug/mask_roi_{side}.png", mask_roi)
    # 2) Extraer contorno
    cnts,_ = cv2.findContours(mask_roi,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    if not cnts: return orig_img
    contour = max(cnts, key=cv2.contourArea).reshape(-1,2).astype(np.float32)
    # 3) PCA
    mean,vecs,_ = cv2.PCACompute2(contour, mean=None)
    axis = vecs[0]
    # 4) Extremos
    dif = contour - mean
    projs = dif.dot(axis.T)
    p1 = list(contour[np.argmin(projs)].astype(int))
    p2 = list(contour[np.argmax(projs)].astype(int))
    if p1[1] > p2[1]: p1, p2 = p2, p1
    
    # Coordenadas punto 1
    p1x = p1[0]
    p1y = p1[1]
    # Coordenadas punto 2
    p2x = p2[0]
    p2y = p2[1]
    
    if (side == "izq") and (p1[0] > p2[0]):
        p1[1] = p2y
        p2[1] = p1y
        
    # Ajustar a coords global
    p1g = (p1[0]+coords[0], p1[1]+coords[1])
    p2g = (p2[0]+coords[0], p2[1]+coords[1])
    # 5) Dibujar
    cv2.circle(orig_img, p1g, 4,(0,255,0),-1)
    cv2.circle(orig_img, p2g, 4,(0,255,0),-1)

    # === Agregar leyenda de los puntos ===
    cv2.putText(orig_img, 'p1', (p1g[0] + 10, p1g[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    cv2.putText(orig_img, 'p2', (p2g[0] + 10, p2g[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    # Extender la línea hasta los límites de la imagen o hasta la línea media
    h, w = orig_img.shape[:2]
    midline_x = inc_center_x if inc_center_x is not None else None
    
    exp1, exp2 = extend_line_to_boundaries(p1g, p2g, w, h, midline_x)
    
    cv2.line(orig_img, exp1, exp2, (0,0,255), 2)
    
    # Para los angulos
    # Coordenada central en X
    center_x = (x1 + x2) // 2
    # Coordenada para poner el texto justo debajo de la caja
    text_y = y2 + 30

    if midline_x is not None:
        midline_top = (midline_x, 0)
        midline_bottom = (midline_x, h)
        intersection = calculate_line_intersection(exp1, exp2, midline_top, midline_bottom)
        
        if intersection:
            cv2.circle(orig_img, intersection, 6, (255, 0, 255), -1)
            angle = calculate_angle(exp1, exp2, midline_x)
            angle_text = f"Angulo: {abs(angle):.1f} Deg"
            cv2.putText(orig_img, angle_text, 
            (center_x - 50, text_y),  # desplazamos 50 px a la izquierda para centrar mejor el texto
            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 255), 2)
            
            radius = 40
            start_angle = 90
            
            if angle < 0:
                end_angle = 90 - abs(angle)
            else:
                end_angle = 90 + abs(angle)
            
            cv2.ellipse(orig_img, intersection, (radius, radius), 
                        0, min(start_angle, end_angle), max(start_angle, end_angle), 
                        (255, 0, 255), 2)
            
            #cv2.putText(orig_img, "Intersección", 
             #          (intersection[0] + 10, intersection[1]), 
              #         cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 255), 2)
        else:
            angle = 0
    return orig_img, angle


# Dibujar todas las detecciones con centros
def draw_box(det):
    if det['name'] not in ['inc', 'canine']:
        x1, y1, x2, y2 = map(int, [det['xmin'], det['ymin'], det['xmax'], det['ymax']])
        color = (0, 255, 255)
        cv2.rectangle(orig, (x1, y1), (x2, y2), color, 2)
        cv2.putText(orig, f"{det['name']}: {det['confidence']:.2f}", 
                   (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        c = get_center(det)
        cv2.circle(orig, c, 5, color, -1)
        
# =================================================================================================
# =================================================================================================
# =================================================================================================
# =================================================================================================
# Bloque principal
# 1. Mejoramiento del Trazado de lineas
# 2. Clasifica las fotos

input_folder = 'img_test'

# Creamos las carpetas de salida si no existen
os.makedirs('Output/Peligro', exist_ok=True)
os.makedirs('Output/No_peligro', exist_ok=True)

image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]

def get_center(detection):
    xmin, ymin, xmax, ymax = map(int, [detection['xmin'], detection['ymin'], detection['xmax'], detection['ymax']])
    cx = (xmin + xmax) // 2
    cy = (ymin + ymax) // 2
    return (cx, cy)

for image_name in image_files:
    img_path = os.path.join(input_folder, image_name)
    img = cv2.imread(img_path)
    
    if img is None:
        print(f"ERROR: No se pudo leer la imagen: {image_name}")
        continue

    print(f"\nProcesando: {image_name}")
    orig = img.copy()
    height, width = img.shape[:2]

    results = model(img)
    df = results.pandas().xyxy[0]

    print("\n=== RESULTADOS DE DETECCIÓN ===")
    print(df)

    incisor_detections = df[df['name'] == 'inc']
    canines = df[df['name'] == 'canine']

    inc_center = None
    if len(incisor_detections) > 0:
        best_inc = incisor_detections.sort_values('confidence', ascending=False).iloc[0]
        inc_center = get_center(best_inc)
        cv2.line(orig, (inc_center[0], 0), (inc_center[0], height), (0, 255, 0), 2)
        cv2.putText(orig, 'Línea media', (inc_center[0] + 10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        print(f"Línea media trazada en x={inc_center[0]}")
    else:
        print("No se detectaron incisivos; no se puede trazar la línea media.")
        continue

    # Lista para almacenar los ángulos de los caninos
    angulos = []

    for _, det in canines.iterrows():
        center = get_center(det)
        side = 'izq' if center[0] < inc_center[0] else 'der'

        x1, y1, x2, y2 = map(int, [det['xmin'], det['ymin'], det['xmax'], det['ymax']])
        color = (255, 0, 0) if side == 'izq' else (0, 0, 255)
        cv2.rectangle(orig, (x1, y1), (x2, y2), color, 2)

        orig, angulo = process_and_draw_canine(det, orig, side, inc_center[0])  # devuelves también el ángulo
        angulos.append(abs(angulo))  # guardamos el ángulo absoluto

        cv2.putText(orig, f"Canino {side}: {det['confidence']:.2f}", 
                    (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        cv2.circle(orig, center, 5, color, -1)

    for _, det in df.iterrows():
        if det['name'] not in ['inc', 'canine']:
            draw_box(det)

    # Decisión final de peligro
    if any(ang > 15 for ang in angulos):
        out_path = os.path.join('Output/Peligro', f"{os.path.splitext(image_name)[0]}_analizada_m3.jpg")
    else:
        out_path = os.path.join('Output/No_peligro', f"{os.path.splitext(image_name)[0]}_analizada_m3.jpg")

    cv2.imwrite(out_path, orig)
    print(f"Guardado en: {out_path}")




Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\josem/.cache\torch\hub\master.zip


C:\Users\josem/.cache\torch\hub\ultralytics_yolov5_master\utils\general.py:32: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources as pkg
YOLOv5  2025-6-16 Python-3.12.3 torch-2.7.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
C:\Users\josem/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Umbral de confianza ajustado a: 0.85

Procesando: foto_1.jpg

=== RESULTADOS DE DETECCIÓN ===
          xmin        ymin         xmax        ymax  confidence  class    name
0  1137.677368  463.714294  1307.943481  780.272339    0.937926      1     inc
1   994.286987  339.954651  1124.865356  613.205200    0.917207      0  canine
2  1336.144165  352.465576  1449.814331  663.742249    0.908585      0  canine
Línea media trazada en x=1222
Guardado en: Output/No_peligro\foto_1_analizada_p.jpg

Procesando: foto_10.jpg

=== RESULTADOS DE DETECCIÓN ===
          xmin        ymin         xmax        ymax  confidence  class    name
0  1169.107300  441.760223  1340.023071  781.725281    0.915781      1     inc
1  1031.080200  338.905762  1154.656616  543.653259    0.873869      0  canine
2  1366.908936  355.143555  1487.561157  544.706177    0.865526      0  canine
Línea media trazada en x=1254


C:\Users\josem/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Guardado en: Output/No_peligro\foto_10_analizada_p.jpg

Procesando: foto_12.jpg

=== RESULTADOS DE DETECCIÓN ===
          xmin        ymin         xmax        ymax  confidence  class    name
0  1145.945923  453.281281  1310.648560  772.125488    0.926675      1     inc
1  1310.315186  339.691681  1424.716064  595.847717    0.903352      0  canine
2  1041.926514  355.357422  1139.601562  604.536438    0.891510      0  canine
Línea media trazada en x=1227


...

Guardado en: Output/Peligro\foto_49_analizada_p.jpg

Procesando: foto_5.jpg

=== RESULTADOS DE DETECCIÓN ===
          xmin        ymin         xmax        ymax  confidence  class    name
0  1121.355713  481.247345  1316.243896  761.721375    0.910407      1     inc
1  1334.326050  391.198853  1451.810425  583.955444    0.883845      0  canine
2   977.313599  399.362244  1089.722778  577.399536    0.879918      0  canine
Línea media trazada en x=1218


C:\Users\josem/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Guardado en: Output/No_peligro\foto_5_analizada_p.jpg

Procesando: foto_6.jpg

=== RESULTADOS DE DETECCIÓN ===
          xmin        ymin         xmax        ymax  confidence  class    name
0  1119.630493  482.308899  1278.567017  793.864075    0.920341      1     inc
1  1292.767578  354.277161  1390.514160  561.963379    0.896552      0  canine
2   993.566345  354.789154  1096.298950  540.383606    0.854132      0  canine
Línea media trazada en x=1198


C:\Users\josem/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Guardado en: Output/Peligro\foto_6_analizada_p.jpg
